# PASO A PASO
1. Importar librerías

In [1]:
import psycopg2
import psycopg2.extras
import pandas as pd
import os 
from pathlib import Path 

2. Generar conexión

In [110]:
connection = psycopg2.connect(
        host= 'localhost',
        dbname = "LCG_Case",
        user= "postgres" ,
        password = 'Lmop-1549f',
        port = 5432)

3. Crear mis cursores

In [111]:
#Selecciono todas mis tablas para después ponerla en un df
Statement1 = "SELECT * FROM BD"
Statement2 = "SELECT * FROM bd_Original"
Statement3 = "SELECT * FROM Departamento"
Statement4 = "SELECT * FROM Vendedor"

In [112]:
cursor = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)
cursor.execute(Statement1.encode('LATIN9'))
BD = cursor.fetchall()
cursor2 = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)
cursor2.execute(Statement2.encode('LATIN9'))
bd_Original = cursor2.fetchall()
cursor3 = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)
cursor3.execute(Statement3.encode('LATIN9'))
Departamentos = cursor3.fetchall()
cursor4 = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)
cursor4.execute(Statement4.encode('LATIN9'))
Vendedor = cursor4.fetchall()
print(type(BD),type(bd_Original),type(Departamentos),type(Vendedor))

<class 'list'> <class 'list'> <class 'list'> <class 'list'>


4. Creo mis df y le cambio el nombre a las columnas

In [113]:
BD = pd.DataFrame(BD)
BD.columns =["Fecha","Número de Vendedor","Nom_Completo_Vendedor","Número de cliente","Tipo","Departamento - Clave","Departamento","Familia - Clave","Familia","Ventas Netas (Q)","Costo"]
BD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7414 entries, 0 to 7413
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Fecha                  7392 non-null   object
 1   Número de Vendedor     7392 non-null   object
 2   Nom_Completo_Vendedor  0 non-null      object
 3   Número de cliente      7392 non-null   object
 4   Tipo                   7392 non-null   object
 5   Departamento - Clave   7387 non-null   object
 6   Departamento           5 non-null      object
 7   Familia - Clave        7392 non-null   object
 8   Familia                7392 non-null   object
 9   Ventas Netas (Q)       7392 non-null   object
 10  Costo                  7392 non-null   object
dtypes: object(11)
memory usage: 637.3+ KB


In [114]:
bd_Original = pd.DataFrame(bd_Original)
bd_Original.columns =["Depto","Departamento","Fam","Familia","anioretail","mesretail","ventasnetas","costos","Utilidad","% Margen"]
bd_Original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4001 entries, 0 to 4000
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Depto         4001 non-null   object
 1   Departamento  4001 non-null   object
 2   Fam           4001 non-null   object
 3   Familia       4001 non-null   object
 4   anioretail    4001 non-null   object
 5   mesretail     4001 non-null   object
 6   ventasnetas   4001 non-null   int64 
 7   costos        4001 non-null   int64 
 8   Utilidad      4001 non-null   int64 
 9   % Margen      4001 non-null   object
dtypes: int64(3), object(7)
memory usage: 312.7+ KB


In [115]:
Departamento = pd.DataFrame(Departamentos)
Departamento.columns =["Departamento - Clave","Departamento"]
Departamento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Departamento - Clave  32 non-null     int64 
 1   Departamento          32 non-null     object
dtypes: int64(1), object(1)
memory usage: 640.0+ bytes


In [116]:
Vendedor = pd.DataFrame(Vendedor)
Vendedor.columns =["No. Vendedor","Nombre","Apellido"]
Vendedor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   No. Vendedor  8 non-null      int64 
 1   Nombre        8 non-null      object
 2   Apellido      8 non-null      object
dtypes: int64(1), object(2)
memory usage: 320.0+ bytes


5. Transformación de mis df

In [117]:
#Generar mi columnas de fechas
BD['Fecha'] = pd.to_datetime(BD['Fecha'])
BD['Año'] = BD['Fecha'].dt.year
BD['Mes'] = BD['Fecha'].dt.month

In [118]:
BD.head(4)

,Fecha,Número de Vendedor,Nom_Completo_Vendedor,Número de cliente,Tipo,Departamento - Clave,Departamento,Familia - Clave,Familia,Ventas Netas (Q),Costo,Año,Mes
0,2015-01-01,7,None,66,Real,21,None,328,TRATAMIENTO DE CONCRETO,31740.12000,22046.54000,2015.0,1.0
1,2015-01-01,5,None,45,Real,21,None,370,PINTURAS DECORATIVAS,2377.30000,1761.35000,2015.0,1.0
2,2015-01-01,8,None,75,Real,None,Pinturas,371,BARNICES ARQUITECTÓNICOS,20422.48000,13189.05000,2015.0,1.0
3,2015-01-01,6,None,57,Real,21,None,372,PINTURA AUTOMOTRIZ,719.80000,668.40000,2015.0,1.0


In [119]:
# Combinar los DataFrames usando la columna 'Número de Vendedor' y 'No. Vendedor'
Vendedor['No. Vendedor'] = Vendedor['No. Vendedor'].astype('string')
merged_df = pd.merge(BD, Vendedor, left_on='Número de Vendedor', right_on='No. Vendedor', how='left')
merged_df['Nombre_Completo'] = merged_df['Nombre'] + ' ' + merged_df['Apellido']

In [120]:
merged_df.head(4)

,Fecha,Número de Vendedor,Nom_Completo_Vendedor,Número de cliente,Tipo,Departamento - Clave,Departamento,Familia - Clave,Familia,Ventas Netas (Q),Costo,Año,Mes,No. Vendedor,Nombre,Apellido,Nombre_Completo
0,2015-01-01,7,None,66,Real,21,None,328,TRATAMIENTO DE CONCRETO,31740.12000,22046.54000,2015.0,1.0,7,Carlos,Vela,Carlos Vela
1,2015-01-01,5,None,45,Real,21,None,370,PINTURAS DECORATIVAS,2377.30000,1761.35000,2015.0,1.0,5,Gonzalo,Aguilar,Gonzalo Aguilar
2,2015-01-01,8,None,75,Real,None,Pinturas,371,BARNICES ARQUITECTÓNICOS,20422.48000,13189.05000,2015.0,1.0,8,Ernesto,Pineda,Ernesto Pineda
3,2015-01-01,6,None,57,Real,21,None,372,PINTURA AUTOMOTRIZ,719.80000,668.40000,2015.0,1.0,6,Paulina,Casanova,Paulina Casanova


In [121]:
#  Combinar los DataFrames usando combine_first
Departamento['Departamento - Clave'] = Departamento['Departamento - Clave'].astype('string')
# Paso 1: Llenar los valores de 'Departamento' basados en 'Departamento - Clave'
merged_df = merged_df.merge(Departamento[['Departamento - Clave', 'Departamento']],
                            how='left',
                            left_on='Departamento - Clave',
                            right_on='Departamento - Clave',
                            suffixes=('_original', '_nuevo'))

In [122]:
# Usar los valores de 'Departamento_original' si están disponibles, de lo contrario, mantener 'Departamento_nuevo'
merged_df['Departamento final'] = merged_df.apply(lambda row: row['Departamento_original'] if pd.notna(row['Departamento_original']) else row['Departamento_nuevo'], axis=1)


In [123]:
# Paso 2: Llenar los valores de 'Departamento - Clave' basados en 'Departamento'
merged_df = merged_df.merge(Departamento[['Departamento - Clave', 'Departamento']],
                            how='left',
                            left_on='Departamento final',
                            right_on='Departamento',
                            suffixes=('_original', '_nuevo'))

In [124]:
merged_df.head(5)

,Fecha,Número de Vendedor,Nom_Completo_Vendedor,Número de cliente,Tipo,Departamento - Clave_original,Departamento_original,Familia - Clave,Familia,Ventas Netas (Q),...,Año,Mes,No. Vendedor,Nombre,Apellido,Nombre_Completo,Departamento_nuevo,Departamento final,Departamento - Clave_nuevo,Departamento
0,2015-01-01,7,None,66,Real,21,None,328,TRATAMIENTO DE CONCRETO,31740.12000,...,2015.0,1.0,7,Carlos,Vela,Carlos Vela,Pinturas,Pinturas,21,Pinturas
1,2015-01-01,5,None,45,Real,21,None,370,PINTURAS DECORATIVAS,2377.30000,...,2015.0,1.0,5,Gonzalo,Aguilar,Gonzalo Aguilar,Pinturas,Pinturas,21,Pinturas
2,2015-01-01,8,None,75,Real,None,Pinturas,371,BARNICES ARQUITECTÓNICOS,20422.48000,...,2015.0,1.0,8,Ernesto,Pineda,Ernesto Pineda,NaN,Pinturas,21,Pinturas
3,2015-01-01,6,None,57,Real,21,None,372,PINTURA AUTOMOTRIZ,719.80000,...,2015.0,1.0,6,Paulina,Casanova,Paulina Casanova,Pinturas,Pinturas,21,Pinturas
4,2015-01-01,7,None,62,Real,21,None,376,TINTES PARA MADERA,15589.78000,...,2015.0,1.0,7,Carlos,Vela,Carlos Vela,Pinturas,Pinturas,21,Pinturas


In [125]:
# Convertir las columnas 'Ventas Netas (Q)' y 'Costo' a tipo float
merged_df['Ventas Netas (Q)'] = merged_df['Ventas Netas (Q)'].astype(float)
merged_df['Costo'] = merged_df['Costo'].astype(float)

In [126]:
# Ajustar el signo de las ventas y costos
merged_df['Ventas Netas (Q)'] = merged_df['Ventas Netas (Q)'].apply(lambda x: abs(x) if x is not None else None)
merged_df['Costo'] = merged_df['Costo'].apply(lambda x: -abs(x) if x is not None else None)

In [128]:
# Realizar la conversión a dólares americanos (considerando 7.5 de ventas/costo = 1 dólar)
merged_df['Ventas en USD'] = merged_df['Ventas Netas (Q)'].apply(lambda x: float(x) / 7.5 if x is not None else None)
merged_df['Costo en USD'] = merged_df['Costo'].apply(lambda x: float(x) / 7.5 if x is not None else None)

In [129]:
merged_df.head(5)

,Fecha,Número de Vendedor,Nom_Completo_Vendedor,Número de cliente,Tipo,Departamento - Clave_original,Departamento_original,Familia - Clave,Familia,Ventas Netas (Q),...,No. Vendedor,Nombre,Apellido,Nombre_Completo,Departamento_nuevo,Departamento final,Departamento - Clave_nuevo,Departamento,Ventas en USD,Costo en USD
0,2015-01-01,7,None,66,Real,21,None,328,TRATAMIENTO DE CONCRETO,31740.12,...,7,Carlos,Vela,Carlos Vela,Pinturas,Pinturas,21,Pinturas,4232.016000,-2939.538667
1,2015-01-01,5,None,45,Real,21,None,370,PINTURAS DECORATIVAS,2377.30,...,5,Gonzalo,Aguilar,Gonzalo Aguilar,Pinturas,Pinturas,21,Pinturas,316.973333,-234.846667
2,2015-01-01,8,None,75,Real,None,Pinturas,371,BARNICES ARQUITECTÓNICOS,20422.48,...,8,Ernesto,Pineda,Ernesto Pineda,NaN,Pinturas,21,Pinturas,2722.997333,-1758.540000
3,2015-01-01,6,None,57,Real,21,None,372,PINTURA AUTOMOTRIZ,719.80,...,6,Paulina,Casanova,Paulina Casanova,Pinturas,Pinturas,21,Pinturas,95.973333,-89.120000
4,2015-01-01,7,None,62,Real,21,None,376,TINTES PARA MADERA,15589.78,...,7,Carlos,Vela,Carlos Vela,Pinturas,Pinturas,21,Pinturas,2078.637333,-1444.670667


In [130]:
# Calcular la utilidad restando 'Ventas en USD' y 'Costo en USD'
merged_df['Utilidad en USD'] = merged_df['Ventas en USD'] + merged_df['Costo en USD']

In [131]:
merged_df.head(4)

,Fecha,Número de Vendedor,Nom_Completo_Vendedor,Número de cliente,Tipo,Departamento - Clave_original,Departamento_original,Familia - Clave,Familia,Ventas Netas (Q),...,Nombre,Apellido,Nombre_Completo,Departamento_nuevo,Departamento final,Departamento - Clave_nuevo,Departamento,Ventas en USD,Costo en USD,Utilidad en USD
0,2015-01-01,7,None,66,Real,21,None,328,TRATAMIENTO DE CONCRETO,31740.12,...,Carlos,Vela,Carlos Vela,Pinturas,Pinturas,21,Pinturas,4232.016000,-2939.538667,1292.477333
1,2015-01-01,5,None,45,Real,21,None,370,PINTURAS DECORATIVAS,2377.30,...,Gonzalo,Aguilar,Gonzalo Aguilar,Pinturas,Pinturas,21,Pinturas,316.973333,-234.846667,82.126667
2,2015-01-01,8,None,75,Real,None,Pinturas,371,BARNICES ARQUITECTÓNICOS,20422.48,...,Ernesto,Pineda,Ernesto Pineda,NaN,Pinturas,21,Pinturas,2722.997333,-1758.540000,964.457333
3,2015-01-01,6,None,57,Real,21,None,372,PINTURA AUTOMOTRIZ,719.80,...,Paulina,Casanova,Paulina Casanova,Pinturas,Pinturas,21,Pinturas,95.973333,-89.120000,6.853333


In [132]:
# Calcular el margen
merged_df['Margen'] = merged_df['Utilidad en USD'] / merged_df['Ventas en USD']

In [133]:
merged_df.head(4)

,Fecha,Número de Vendedor,Nom_Completo_Vendedor,Número de cliente,Tipo,Departamento - Clave_original,Departamento_original,Familia - Clave,Familia,Ventas Netas (Q),...,Apellido,Nombre_Completo,Departamento_nuevo,Departamento final,Departamento - Clave_nuevo,Departamento,Ventas en USD,Costo en USD,Utilidad en USD,Margen
0,2015-01-01,7,None,66,Real,21,None,328,TRATAMIENTO DE CONCRETO,31740.12,...,Vela,Carlos Vela,Pinturas,Pinturas,21,Pinturas,4232.016000,-2939.538667,1292.477333,0.305405
1,2015-01-01,5,None,45,Real,21,None,370,PINTURAS DECORATIVAS,2377.30,...,Aguilar,Gonzalo Aguilar,Pinturas,Pinturas,21,Pinturas,316.973333,-234.846667,82.126667,0.259096
2,2015-01-01,8,None,75,Real,None,Pinturas,371,BARNICES ARQUITECTÓNICOS,20422.48,...,Pineda,Ernesto Pineda,NaN,Pinturas,21,Pinturas,2722.997333,-1758.540000,964.457333,0.354190
3,2015-01-01,6,None,57,Real,21,None,372,PINTURA AUTOMOTRIZ,719.80,...,Casanova,Paulina Casanova,Pinturas,Pinturas,21,Pinturas,95.973333,-89.120000,6.853333,0.071409


In [134]:
merged_df.columns

Index(['Fecha', 'Número de Vendedor', 'Nom_Completo_Vendedor',
       'Número de cliente', 'Tipo', 'Departamento - Clave_original',
       'Departamento_original', 'Familia - Clave', 'Familia',
       'Ventas Netas (Q)', 'Costo', 'Año', 'Mes', 'No. Vendedor', 'Nombre',
       'Apellido', 'Nombre_Completo', 'Departamento_nuevo',
       'Departamento final', 'Departamento - Clave_nuevo', 'Departamento',
       'Ventas en USD', 'Costo en USD', 'Utilidad en USD', 'Margen'],
      dtype='object')

In [135]:
Final_df = merged_df[['Fecha','Año','Mes','No. Vendedor','Nombre_Completo','Número de cliente','Tipo','Departamento - Clave_nuevo','Departamento final','Familia - Clave','Familia','Ventas Netas (Q)','Costo','Ventas en USD', 'Costo en USD', 'Utilidad en USD', 'Margen']]

In [140]:
Final_df[['Ventas Netas (Q)', 'Costo', 'Ventas en USD', 'Costo en USD', 'Utilidad en USD', 'Margen']] = \
    Final_df[['Ventas Netas (Q)', 'Costo', 'Ventas en USD', 'Costo en USD', 'Utilidad en USD', 'Margen']].round(2)

C:\Users\calde\AppData\Local\Temp\ipykernel_16692\3930090258.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Final_df[['Ventas Netas (Q)', 'Costo', 'Ventas en USD', 'Costo en USD', 'Utilidad en USD', 'Margen']] = \


In [141]:
Final_df.head(5)

,Fecha,Año,Mes,No. Vendedor,Nombre_Completo,Número de cliente,Tipo,Departamento - Clave_nuevo,Departamento final,Familia - Clave,Familia,Ventas Netas (Q),Costo,Ventas en USD,Costo en USD,Utilidad en USD,Margen
0,2015-01-01,2015.0,1.0,7,Carlos Vela,66,Real,21,Pinturas,328,TRATAMIENTO DE CONCRETO,31740.12,-22046.54,4232.02,-2939.54,1292.48,0.31
1,2015-01-01,2015.0,1.0,5,Gonzalo Aguilar,45,Real,21,Pinturas,370,PINTURAS DECORATIVAS,2377.30,-1761.35,316.97,-234.85,82.13,0.26
2,2015-01-01,2015.0,1.0,8,Ernesto Pineda,75,Real,21,Pinturas,371,BARNICES ARQUITECTÓNICOS,20422.48,-13189.05,2723.00,-1758.54,964.46,0.35
3,2015-01-01,2015.0,1.0,6,Paulina Casanova,57,Real,21,Pinturas,372,PINTURA AUTOMOTRIZ,719.80,-668.40,95.97,-89.12,6.85,0.07
4,2015-01-01,2015.0,1.0,7,Carlos Vela,62,Real,21,Pinturas,376,TINTES PARA MADERA,15589.78,-10835.03,2078.64,-1444.67,633.97,0.30


In [137]:
Final_df.shape

(7414, 17)

In [142]:
root_dir = Path(".").resolve().parent
out_name = "reporte.csv"  
out_path = os.path.join(root_dir,"Datos","Processed", out_name)
Final_df.to_csv(out_path,encoding='latin9', index=False)